## создание сводных для кажого бренда и фасовки, нужно сделать вычисления в этой таблице и соединить их вместе

In [1]:
import pandas as pd

### Бренды

In [2]:
brands = [
    'BlackBurn',
    'Burn',
    'B3',
    'Banger',
    'PeterRalf',
    'Overdose'
]

### Планы на каждый бренд

In [3]:
BB_Plan = 50000
B_Plan = 2350
B3_Plan = 327
BNG_Plan = 3450
PR_Plan = 80
OVD_Plan = 7650

### Вычисление доли фасовок и вычисление плана

In [4]:
df = pd.read_excel(r"D:\Analysis Burn\Кирилл Кузема\Планирование июль 23_подготовка.xlsx", sheet_name='Исходные данные')

In [5]:
# создание для каждого бренда таблицы с долей граммовки
for i in brands:
    exec(f"{i} = df.loc[df['Тип'] == '{i}'].groupby( \
        by=['Вес (гр)'] \
    )['Кг'].sum().reset_index()" )

    exec(f"{i}['доля'] = {i}['Кг'] / {i}['Кг'].sum() ")

    
    if i == 'BlackBurn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BB_Plan}")

    if i == 'Burn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B_Plan}")

    if i == 'B3':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B3_Plan}")

    if i == 'Banger':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BNG_Plan}")
        
    if i == 'PeterRalf':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {PR_Plan}")

    if i == 'Overdose':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {OVD_Plan}")


### создание таблиц для каждого бренда каждой фасовки

In [6]:
brands_dict = {
    'BlackBurn' : BlackBurn, 
    'Burn' : Burn,
    'B3' : B3, 
    'Banger' : Banger,
    'PeterRalf': PeterRalf, 
    'Overdose': Overdose
}

In [7]:
dfs = {}
for brand in brands:
   data = brands_dict[brand]
   for idx, row in data.iterrows():
      need_row = int(row['Вес (гр)'])
      dfs[f"{brand}_{str(need_row)}"] = df.loc[(df['Тип'] == brand) & (df['Вес (гр)'] == need_row)]

### Создание таблицы с планированием

In [21]:
pivot_tables = {}

for data in dfs:

    dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
    dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month

    pivot_tables[data] = dfs.get(data).pivot_table(
        values=['Кг'],
        index=['Тип', 'Вес (гр)','Товар\вкус'],
        columns=['year', 'month'], 
        aggfunc="sum"
    ).round().reset_index()


    current_brand = pivot_tables[data]['Тип'][0]
    current_group = pivot_tables[data]['Вес (гр)'][0]

    for br in brands_dict:
        if br == current_brand:
            current_plan = brands_dict[br].loc[brands_dict[br]['Вес (гр)'] == current_group]['планируемый_объем'].iloc[0]
    

    # сделать двухэтажные заголовки обычными
    pivot_tables[data].columns = ['_'.join([str(x) for x in idx if x and x != 'Кг']) for idx in pivot_tables[data].columns]

    pivot_tables[data]['Последний месяц'] = pivot_tables[data].iloc[:,-1:]

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Средний объем в месяц при наличии'] = pivot_tables[data].median(axis=1, numeric_only=True)

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Наибольший объем в месяц'] = pivot_tables[data].max(axis=1, numeric_only=True)

    # добавить колонку Доля продаж исходя из ср-зн
    pivot_tables[data]['Доля продаж исходя из ср-зн'] = ( pivot_tables[data]['Средний объем в месяц при наличии'] / pivot_tables[data]['Средний объем в месяц при наличии'].sum() )

    # добавить колонку Планируемый объем
    pivot_tables[data]['Планируемый объем'] = current_plan * pivot_tables[data]['Доля продаж исходя из ср-зн']
    
    # добавить колонку Дополнительно
    pivot_tables[data]['Дополнительно'] = pivot_tables[data]['Планируемый объем'] * 0.1
    
    # добавить колонку Итоговый заказ на месяц
    pivot_tables[data]['Итоговый заказ на месяц'] = pivot_tables[data]['Дополнительно'] + pivot_tables[data]['Планируемый объем']

    # добавить колонку Итоговый заказ на следующий месяц
    pivot_tables[data]['Итоговый заказ на месяц вперед'] = pivot_tables[data]['Итоговый заказ на месяц'] * 1.1

    # добавить колонку Итоговый заказ на два месяца вперед
    pivot_tables[data]['Итоговый заказ на два месяца вперед'] = pivot_tables[data]['Итоговый заказ на месяц вперед'] * 1.1

    # добавить колонку Разница по отношению к прошлому месяцу 
    pivot_tables[data]['Разница по отношению к прошлому месяцу'] = pivot_tables[data]['Итоговый заказ на месяц'] - pivot_tables[data]['Последний месяц']

    pivot_tables[data].sort_values(by=['Доля продаж исходя из ср-зн'], ascending = False, inplace=True)

    pivot_tables[data]['Доля продаж исходя из ср-зн'] = pivot_tables[data]['Доля продаж исходя из ср-зн'] * 100

    pivot_tables[data].iloc[:,-10:] = pivot_tables[data].iloc[:,-10:].round(2)
    

C:\Users\User\AppData\Local\Temp\ipykernel_16988\733067782.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
C:\Users\User\AppData\Local\Temp\ipykernel_16988\733067782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_16988\733067782.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [9]:
BlackBurn

,Вес (гр),Кг,доля,планируемый_объем
0,25,205489.34,0.320831,16041.531783
1,100,99551.30,0.155430,7771.475362
2,200,335451.00,0.523740,26186.992854


In [219]:
pivot_tables['BlackBurn_25'].to_excel(r"C:\Users\User\Desktop\test.xlsx")

In [54]:
prepared_df = pd.DataFrame()
for i in pivot_tables:
    if prepared_df.empty == True:
        prepared_df = pivot_tables[i]
    else:
        prepared_df = pd.concat([prepared_df, pivot_tables[i]])

In [57]:
prepared_df.to_excel(r"C:\Users\User\Desktop\test.xlsx", index=False)

In [32]:
pivot_tables['BlackBurn_25'].iloc[:,-10:] = pivot_tables['BlackBurn_25'].iloc[:,-10:].round(2)

In [33]:
pivot_tables['BlackBurn_25']

,Тип,Вес (гр),Товар\вкус,2022_1,2022_2,2022_3,2022_4,2022_5,2022_6,2022_7,...,Последний месяц,Средний объем в месяц при наличии,Наибольший объем в месяц,Доля продаж исходя из ср-зн,Планируемый объем,Дополнительно,Итоговый заказ на месяц,Итоговый заказ на месяц вперед,Итоговый заказ на два месяца вперед,Разница по отношению к прошлому месяцу
49,BlackBurn,25,Raspberry Shock (Кислая малина),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1060.0,1060.0,1060.0,7.30,1170.31,117.03,1287.34,1416.08,1557.68,227.34
67,BlackBurn,25,На Чиле (Тропический Сок),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,860.0,860.0,1826.0,5.92,949.50,94.95,1044.45,1148.89,1263.78,184.45
39,BlackBurn,25,Nutella (Шоколадно-ореховая паста),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,848.0,848.0,848.0,5.84,936.25,93.62,1029.87,1132.86,1246.15,181.87
66,BlackBurn,25,На Расслабоне (Банановое Суфле),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,701.0,701.0,1742.0,4.82,773.95,77.40,851.35,936.48,1030.13,150.35
27,BlackBurn,25,Ice Baby (Ягодный Сорбет С Грейпфрутом) (Colla...,441.0,456.0,636.0,318.0,492.0,596.0,559.0,...,629.0,492.0,678.0,3.39,543.20,54.32,597.52,657.27,723.00,-31.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,BlackBurn,25,Almond Icecream (Миндальное Мороженое),82.0,42.0,88.0,45.0,60.0,80.0,74.0,...,82.0,61.0,90.0,0.42,67.35,6.73,74.08,81.49,89.64,-7.92
47,BlackBurn,25,Pistachio Ice Snow (Фисташковое Мороженое),91.0,48.0,77.0,34.0,55.0,68.0,78.0,...,107.0,56.0,107.0,0.39,61.83,6.18,68.01,74.81,82.29,-38.99
4,BlackBurn,25,Asian Lychee (Личи),61.0,35.0,87.0,36.0,33.0,62.0,67.0,...,77.0,54.0,87.0,0.37,59.62,5.96,65.58,72.14,79.35,-11.42
0,BlackBurn,25,After8 (Шоколад-Мята),72.0,36.0,83.0,42.0,48.0,38.0,73.0,...,70.0,50.0,83.0,0.34,55.20,5.52,60.72,66.80,73.48,-9.28
